# packages

In [496]:
import subprocess
import sys

def install(package):
    subprocess.check_call([sys.executable, "-m", "pip", "install", package])

install("pandas")
install("numpy")
install("matplotlib")

# imports

In [497]:
import pandas as pd
import numpy as np
import json
import csv
import os

# Read Extraction Data from CSV

In [498]:
df = pd.read_csv(filepath_or_buffer="data/04 extraction consolidation results_3.csv",sep="|")

In [499]:
df

,Title,DT Definition,virtual space,Kritzinger,DT is specific,modeling language original,note on modeling language,model type original,modeling language,language fidelity category,...,DTPurposeCategory,use case domain,system lifecycle phase,twinning target,twin lifecycle phase,purpose /expected benefit of using models,open challenges,Technological Readiness Level,notes,Use Case
0,A framework for Model-Driven Engineering of re...,"dynamic virtual model of a system, processor, ...",Model of PT,NaN,NaN,SysML,NaN,structure,SysML,NaN,...,Monitoring,Transportation and Storage,Design,Individual Systems,As-Designed,formal specification of information in the for...,NaN,TRL 1-3,NaN,limited traffic zone access control
1,A framework for Model-Driven Engineering of re...,NaN,NaN,NaN,NaN,Knowledge base model,contribution of the paper,NaN,Knowledge base model,NaN,...,NaN,NaN,Design,NaN,As-Designed,NaN,NaN,NaN,NaN,NaN
2,A Methodology for Digital Twin Modeling and De...,"""The digital twin (DT) is a virtual representa...",Model of PT,NaN,NaN,AutomationML,NaN,structure,AutomationML,NaN,...,Synchronization,Manufacturing,Design,Individual Systems,As-Designed,"model the physical device, serving as the virt...",NaN,TRL 1-3,NaN,refinery automationsystem with four valves
3,A Methodology for Digital Twin Modeling and De...,NaN,NaN,NaN,NaN,NaN,NaN,structure,NaN,NaN,...,NaN,NaN,Operate,Systems of Systems,As-Operated,NaN,NaN,NaN,NaN,NaN
4,A Novel Methodology to Construct Digital Twin ...,"""a digital twin of a system consists of a set ...",Comprehensive,DS,NaN,Fault Tree,NaN,behavior,Fault Tree,NaN,...,NaN,"Professional, Scientific and Technical Activities",Operate,Individual Systems,As-Operated,NaN,NaN,TRL 1-3,NaN,ground station of a spacecraft
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75,TwinOps - DevOps Meets Model-Based Engineering...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,Implement,NaN,As-Designed,NaN,NaN,NaN,NaN,NaN
76,TwinOps - DevOps Meets Model-Based Engineering...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,Implement,NaN,As-Designed,NaN,NaN,NaN,NaN,NaN
77,Understanding and Improving Model-Driven IoT S...,"""A digital twin (DT) of a system consists of a...",Comprehensive,DS,NaN,MontiThings,NaN,structure,MontiThings,NaN,...,Collect Data?,Accommodation,Operate,Individual Systems,As-Operated,NaN,NaN,TRL 1-3,NaN,"smart lawn watering system; heating, ventilati..."
78,Understanding and Improving Model-Driven IoT S...,NaN,NaN,NaN,NaN,NaN,NaN,structure,NaN,NaN,...,NaN,NaN,Implement,NaN,As-Operated,NaN,NaN,NaN,NaN,NaN


In [500]:
result = df['Title'].to_list()

In [501]:
result = list(dict.fromkeys(result))
print(len(result))

34


In [502]:
titles = list(df.columns)

In [503]:
metadata = pd.read_csv("./data/search_results.csv", sep=";", encoding="latin-1")

In [504]:
metadata = metadata.drop_duplicates(subset=['Title'])

In [505]:
data = df.merge(metadata, how='left', on='Title')
#unique function

In [506]:
del metadata
del df
del titles

In [507]:
index = 0
print('{', end="")
for row in data:
    print(index, row)
    if index == 0:
        #print('\''+row+'\'', end="")
        index += 1
        continue
    #print(', '+'\''+row+'\'', end="")
    index += 1
print('}')

{0 Title
1 DT Definition
2 virtual space
3 Kritzinger
4 DT is specific
5 modeling language original
6 note on modeling language
7 model type original
8 modeling language
9 language fidelity category
10 language content category
11 language content category consolidated
12 model type
13 model processing technique
14 model usage technique
15 source
16 source category
17 source sub-category 1
18 source model target category
19 target
20 target category
21 target sub-category
22 target model target category
23 purpose of MDE application
24 purpose of DT
25 DTPurposeCategory
26 use case domain
27 system lifecycle phase
28 twinning target
29 twin lifecycle phase
30 purpose /expected benefit of using models
31 open challenges
32 Technological Readiness Level
33 notes
34 Use Case
35 Publication Title
36 Authors
37 Publication Year
38 Document Identifier
39 Publisher
}


# create JSON from CSV

In [508]:
model_trafos = 'modelTrafos'
modelling_language = 'modeling language'

In [509]:
keys = ['Title', 'DT Definition', 'virtual space', 'Kritzinger', 'DT is specific', 'modeling language original', 'note on modeling language', 'model type original', 'modeling language', 'language fidelity category', 'language content category', 'language content category consolidated', 'model type', 'model processing technique', 'model usage technique', 'source', 'source category', 'source sub-category 1', 'source model target category', 'target', 'target category', 'target sub-category', 'target model target category', 'purpose of MDE application', 'purpose of DT', 'DTPurposeCategory', 'use case domain', 'system lifecycle phase', 'twinning target', 'twin lifecycle phase', 'purpose /expected benefit of using models', 'open challenges', 'Technological Readiness Level', 'notes', 'Use Case', 'Publication Title', 'Authors', 'Publication Year', 'Document Identifier', 'Publisher']

In [510]:
out_menge_1 = {}
arrayindexes = [1,2,3,4,23,24,26,28,29,30,31,32,33,34,35,36,37,38,39]
for key in result:
    out_menge_1[key] = {**{modelling_language:{}, model_trafos:{}}, **{keys[x]: None for x in arrayindexes}}
    
    
x1 = {keys[1]:None,
                        keys[2]:None,
                        keys[3]:None,
                        keys[4]:None,
                        keys[15]:None,
                        keys[16]:None,
                        keys[17]:None,
                        keys[19]:None, 

                        model_trafos:{},
                        keys[22]:None,
                        keys[23]:None, 
                        keys[24]:None, 
                        keys[25]:None, 
                        modelling_language:{},
                        
                        }

print(out_menge_1)

{'A framework for Model-Driven Engineering of resilient software-controlled systems': {'modeling language': {}, 'modelTrafos': {}, 'DT Definition': None, 'virtual space': None, 'Kritzinger': None, 'DT is specific': None, 'purpose of MDE application': None, 'purpose of DT': None, 'use case domain': None, 'twinning target': None, 'twin lifecycle phase': None, 'purpose /expected benefit of using models': None, 'open challenges': None, 'Technological Readiness Level': None, 'notes': None, 'Use Case': None, 'Publication Title': None, 'Authors': None, 'Publication Year': None, 'Document Identifier': None, 'Publisher': None}, 'A Methodology for Digital Twin Modeling and Deployment for Industry 4.0': {'modeling language': {}, 'modelTrafos': {}, 'DT Definition': None, 'virtual space': None, 'Kritzinger': None, 'DT is specific': None, 'purpose of MDE application': None, 'purpose of DT': None, 'use case domain': None, 'twinning target': None, 'twin lifecycle phase': None, 'purpose /expected benef

In [511]:
data.reset_index()
for title in result:
    rownum = 0
    
    for index, row in data.iterrows():
        if title == row['Title'] and not pd.isna(title):
            rowcopy = row.__deepcopy__()
            
            if row['DT Definition'] is not None and not pd.isna(row['DT Definition']):
                out_menge_1[title]['DT Definition'] = row['DT Definition']
                
                if row['virtual space'] is not None and not pd.isna(row['virtual space']):
                    out_menge_1[title]['virtual space'] = row['virtual space']
            
                if row['Kritzinger'] is not None and not pd.isna(row['Kritzinger']):
                    out_menge_1[title]['Kritzinger'] = row['Kritzinger']
            
                if row['DT is specific'] is not None and not pd.isna(row['DT is specific']):
                    out_menge_1[title]['DT is specific'] = row['DT is specific']

            if row['purpose of DT'] is not None and not pd.isna(row['purpose of DT']):
                out_menge_1[title]['purpose of DT'] = row['purpose of DT']

            if row['DTPurposeCategory'] is not None and not pd.isna(row['DTPurposeCategory']):
                out_menge_1[title]['DTPurposeCategory'] = row['DTPurposeCategory']

            if row['use case domain'] is not None and not pd.isna(row['use case domain']):
                out_menge_1[title]['use case domain'] = row['use case domain']
            
            if row['twinning target'] is not None and not pd.isna(row['twinning target']):
                if out_menge_1[title]['twinning target'] is None:
                    out_menge_1[title]['twinning target'] = row['twinning target']
                else:
                    out_menge_1[title]['twinning target'] += ";"
                    out_menge_1[title]['twinning target'] +=  row['twinning target']

            if row['purpose /expected benefit of using models'] is not None and not pd.isna(row['purpose /expected benefit of using models']):
                out_menge_1[title]['purpose /expected benefit of using models'] = row['purpose /expected benefit of using models']

            if row['open challenges'] is not None and not pd.isna(row['open challenges']):
                out_menge_1[title]['open challenges'] = row['open challenges']

            if row['Technological Readiness Level'] is not None and not pd.isna(row['Technological Readiness Level']):
                out_menge_1[title]['Technological Readiness Level'] = row['Technological Readiness Level']
            
            if row['notes'] is not None and not pd.isna(row['notes']):
                out_menge_1[title]['notes'] = row['notes']

            if row['Use Case'] is not None and not pd.isna(row['Use Case']):
                out_menge_1[title]['Use Case'] = row['Use Case']

            if row['Publication Title'] is not None and not pd.isna(row['Publication Title']):
                out_menge_1[title]['Publication Title'] = row['Publication Title']
            
            if row['Authors'] is not None and not pd.isna(row['Authors']):
                out_menge_1[title]['Authors'] = row['Authors']
            
            if row['Publication Year'] is not None and not pd.isna(row['Publication Year']):
                out_menge_1[title]['Publication Year'] = row['Publication Year']

            if row['Document Identifier'] is not None and not pd.isna(row['Document Identifier']):
                out_menge_1[title]['Document Identifier'] = row['Document Identifier']

            if row['Publisher'] is not None and not pd.isna(row['Publisher']):
                out_menge_1[title]['Publisher'] = row['Publisher']
                
            if row['twin lifecycle phase'] is not None and not pd.isna(row['twin lifecycle phase']):
                out_menge_1[title]['twin lifecycle phase'] = row['twin lifecycle phase']
            
            if row['modeling language'] is not None and not pd.isna(row['modeling language']):
                if out_menge_1[title]['modeling language'] is None:
                    out_menge_1[title]['modeling language'] = {}
                out_menge_1[title]['modeling language'][row['modeling language']] = {'fidelity':row['language fidelity category'],'category': row['language content category'],'consolidated_category':row['language content category consolidated'],'model_type':row['model type']}#.append(row['modeling language'])

            # multiple Trafos
            arrayindexes = [0,1,2,3,4,23,24,26,28,29,30,31,32,33,34,35,36,37,38,39]
            rowjson = rowcopy.drop(row.iloc[arrayindexes].index).to_json()
            try:
                out_menge_1[title][model_trafos][rownum] = rowjson
            except Exception as ex:
                out_menge_1[title][model_trafos] = {}
                out_menge_1[title][model_trafos][rownum] = rowjson
            #out_menge_1[title]['data'][rownum].append(rowjson)
            rownum+=1


In [512]:
out_menge_1

{'A framework for Model-Driven Engineering of resilient software-controlled systems': {'modeling language': {'SysML': {'fidelity': nan,
    'category': 'ontology',
    'consolidated_category': 'logical or physical',
    'model_type': 'Structure'},
   'Knowledge base model': {'fidelity': nan,
    'category': 'behavior',
    'consolidated_category': 'structure or behavior',
    'model_type': '??'}},
  'modelTrafos': {0: '{"modeling language original":"SysML","note on modeling language":null,"model type original":"structure","modeling language":"SysML","language fidelity category":null,"language content category":"ontology","language content category consolidated":"logical or physical","model type":"Structure","model processing technique":"Model to Model Transformation","model usage technique":null,"source":"SysML: BDD","source category":"behavior model","source sub-category 1":"state-based","source model target category":null,"target":"Knowledge Base of software digital twins (Metamodel 

In [513]:
del rowjson
del rowcopy
del result
del rownum

# export JSON to /json/data_consolidated_semistructured.json

In [514]:
json_object = json.dumps(out_menge_1, indent = 4) 
try:
    os.mkdir('target/')
    os.mkdir('target/img/')
except OSError as error:
    print(error)
with open("target/json/data_consolidated_semistructured__3.json", "w") as outfile:
    outfile.write(json_object)

[WinError 183] Eine Datei kann nicht erstellt werden, wenn sie bereits vorhanden ist: 'target/'
